Credit to https://github.com/zhixuhao/unet for the Unet Model

# Imports

In [ ]:

# !pip install rasterio #used in Google Colab to install the library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
%tensorflow_version 1.x
import numpy as np
import os
import rasterio
import tensorflow as tf
import random
import pandas as pd
from itertools import chain
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, GlobalMaxPool2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import time
import skimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

'''Lines used in Google Colab to load files'''
# from google.colab import drive
# from google.colab import files
# drive.mount('/content/gdrive')

# Functions (normalize, create_dataset, image_tiles)

In [ ]:
def normalize(array):
  return (array - array.min()) / (array.max() - array.min())

def create_dataset(im, dim_size,stride):
    
  no_bands = len(im.indexes)
  data = im.read()
  bands = np.zeros([no_bands,im.shape[0],im.shape[1]])
  for k in range(no_bands):
    temp = data[k,:,:]
    bands[k,:,:] = normalize(temp)
      
  global rows
  global columns
  rows = int((im.shape[0]-dim_size)/stride) + 1
  columns = int((im.shape[1]-dim_size)/stride) + 1

  dataset= np.zeros((rows*columns,dim_size,dim_size,no_bands))
  image_num = 0 
  y_min = 0
  y_max = dim_size
  global orig_width
  global orig_height
  orig_height = im.shape[0]
  orig_width = im.shape[1]
  while (y_max)<orig_height:
    x_min = 0
    x_max = dim_size
    while (x_max)<orig_width:
      for k in range(no_bands):
        temp = bands[k,y_min:y_max,x_min:x_max]
        dataset[image_num,:,:,k] = temp
      image_num = image_num + 1
      x_min = x_min + stride
      x_max = x_max + stride
        
    y_min = y_min + stride
    y_max = y_max + stride

  return dataset,no_bands

def image_tiles(path, image_size, stride, mask_name):
  # Creating tiles for images
  flag = True
  test_counter = 0
  for f in os.listdir(path):
    extension = os.path.splitext(f)[1]
    if extension == '.tif':
      # print("Processed " + f)
      im = rasterio.open(path+f)
      mask_path = './gdrive/My Drive/Jupyter Dataset/'+mask_name
      mask = rasterio.open(mask_path)
      [tempy, no_labels] = create_dataset(mask, image_size, stride)
      [tempx, no_bands] = create_dataset(im, image_size, stride)
      if flag:
        flag = False
        x = tempx
        y = tempy
      else:
        x = np.concatenate([x, tempx],0)
        y = np.concatenate([y, tempy],0)
    test_counter += 1

  print('Number of images generated: {} \nNumber of labels generated: {}'.format(x.shape[0],y.shape[0]))
  del tempx
  del tempy
  del im
  del mask
  return x, y, no_bands, no_labels

# Dataset Creation

In [ ]:
image_size = 128 # New image size
stride = 64 # Stride
data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A/"
# data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A_elev/"
# data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A_srtm/"
# data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A_dem/"
# data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A_NDVI/"
# data_dir = "./gdrive/My Drive/Jupyter Dataset/Clipped/2A_elev_NDVI/"

train_image_path = data_dir + "Train/8/" 
[x_train, y_train, no_bands, no_labels] = image_tiles(train_image_path,image_size,stride,'corrected_new_corine_clipped_mask_final.tif')

test_image_path = data_dir + "Test/" 
[x_valid, y_valid, no_bands, no_labels] = image_tiles(test_image_path,image_size,stride,'corrected_new_corine_clipped_mask_final.tif')
no_classes = no_labels

# U-net Model

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def get_unet(input_img, no_classes, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)

    c4i = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    p4i = MaxPooling2D(pool_size=(2, 2)) (c4i)
    p4i = Dropout(dropout)(p4i)
    
    c5 = conv2d_block(p4i, n_filters=n_filters*32, kernel_size=3, batchnorm=batchnorm)

    u6i = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c5)
    u6i = concatenate([u6i, c4i])
    u6i = Dropout(dropout)(u6i)
    c6i = conv2d_block(u6i, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c6i)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)


    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(no_classes, (1, 1), activation='softmax') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

# Model Compilation and Parameter selection

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR) # Supressing Warnings
input_img = Input((image_size, image_size, no_bands), name='img')
model = get_unet(input_img, no_classes, n_filters=16, batchnorm=True)
model.compile(optimizer=Adam(),
              loss="categorical_crossentropy", metrics=["accuracy"])

# Model Training

In [ ]:
epochs = 50
batch_size = 64
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_valid, y_valid))

# Accuracy Plot

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
ax = plt.figure(figsize = (16,6)).gca()
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim((0,1))
plt.xlim((0,epochs))
ax.xaxis.set_major_locator(MaxNLocator(nbins=epochs/5, integer=True))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plot_name = data_dir[42:-1]+'_plot_1.png'
plt.savefig(plot_name, bbox_inches='tight',dpi=300)

# Loss Plot 

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
ax = plt.figure(figsize = (16,6)).gca()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
# plt.ylim((0,1))
plt.xlim((0,epochs))
ax.xaxis.set_major_locator(MaxNLocator(nbins=epochs/5, integer=True))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plot_name = data_dir[42:-1]+'_plot_2.png'
plt.savefig(plot_name, bbox_inches='tight',dpi=300)

# Defining Image reconstruction

In [ ]:
def image_reconstruction(data, dim_size, stride, rows=rows, columns=columns):
  no_original_images = int(data.shape[0]/(rows*columns))
  var1 = data.shape[0]
  original_images = {}
  im_num = 0
  if dim_size == stride:
    for im in range(no_original_images):
      og_image= []
      flag2 = True
      for c in range(rows):
        row = []
        flag = True
        for p in range(columns):
          # plt.figure()
          # plt.imshow(data[im_num,:,:,0],cmap='gray')
          if flag:
            temp = data[im_num,:,:,:]
            row = temp
            flag = False
          else:
            temp = data[im_num,:,:,:]
            row = np.concatenate((row,temp), axis=1)
          im_num = im_num + 1
        if flag2:
          og_image = row
          flag2 = False
        else:
          og_image = np.concatenate((og_image, row), axis=0)
      original_images[im] = og_image

  else:
    for im in range(no_original_images):
      og_image= []
      y_max=dim_size
      while y_max<orig_height and im_num<var1:
          row = []
          x_max=dim_size
          flag=True
          while x_max<orig_width and im_num<var1:
            if flag==True and y_max==dim_size and x_max==dim_size:
              temp = data[im_num,:,:,:]
              row = temp
              flag=False
            if flag==True and y_max>dim_size:
              temp = data[im_num,stride:,:,:]
              row = temp
              flag=False
            if flag==False and y_max==dim_size and x_max>dim_size:
              temp = data[im_num,:,stride:,:]
              row = np.concatenate((row,temp), axis=1)
            if flag==False and y_max>dim_size and x_max>dim_size:
              temp = data[im_num,stride:,stride:,:]
              row = np.concatenate((row,temp), axis=1)
            x_max = x_max + stride
            im_num = im_num + 1
          if y_max==dim_size:
            og_image = row
          else:
            og_image = np.concatenate((og_image, row), axis=0)
          y_max = y_max + stride
          original_images[im] = og_image

  return original_images

# Prediction and image reconstruction

In [ ]:
result = model.predict(x_valid)
for i in range(result.shape[3]):
  result[:,:,:,i] = result[:,:,:,i] > 0.5
result_final = image_reconstruction(result, image_size, stride)
y_valid_final = image_reconstruction(y_valid, image_size, stride)
im_result = result_final[3] #prediction for the first image in the test dataset
im_y_valid = y_valid_final[3]


# Plotting predictions for every class 

In [ ]:
class_names =['Disc. Urban Fabric','Bare Rocks','Coniferous Forest',
              'Mixed Forest','Natural Grasslands','Olive Groves',
              'Pastures','Principally Agriculture','Schlerophyllus Vegetation',
              'Sea','Sparsely Veg. Areas','No Data']

plt.style.use("ggplot")
plt.style.use('default')
fig, ax = plt.subplots(no_labels,2,figsize=(30,50))
for i in range(no_labels):
  ax[i,0].set_title('Prediction - '+ class_names[i])
  ax[i,0].imshow(im_result[:,:,i], cmap='gray')
  ax[i,0].axis('off')
  ax[i,1].set_title('Original - '+ class_names[i])
  ax[i,1].imshow(im_y_valid[:,:,i], cmap='gray')
  ax[i,1].axis('off')
plt.tight_layout()

plot_name = data_dir[42:-1]+'_plot_3.png'
fig.savefig(plot_name, bbox_inches='tight',dpi=600)

# Confusion Matrix Function

In [ ]:
def calc_conf_mat(px_min,px_max,py_min,py_max,im_result=im_result,im_y_valid=im_y_valid,class_names=class_names):
  conf_mat = np.zeros((im_result.shape[2],im_y_valid.shape[2]),dtype=int)
  result_classes = np.argmax(im_result,axis=2)
  y_valid_classes = np.argmax(im_y_valid,axis=2)
  test_counter = 0
  for y in range(px_min,px_max):
    for x in range(py_min,py_max):
      conf_mat[y_valid_classes[x,y], result_classes[x,y]] = conf_mat[y_valid_classes[x,y],result_classes[x,y]]+ 1
      test_counter += 1
  # print(test_counter)
  # print(conf_mat.sum())
  # print('')  
  df = pd.DataFrame(conf_mat, columns=class_names, index=class_names)
  # print(' '*70,'-- Confusion Matrix for polygon 1 --')
  # print('Columns: Ground Truth')
  # print('Rows: Prediction')
  temp = []
  for i in df:
    temp.append(df[i].sum())
  temp2 = []
  for i in range(conf_mat.shape[0]):
    temp2.append(conf_mat[i].sum())
  temp2.append(np.sum(temp2))
  df.loc['Total'] = temp
  df['Total'] = temp2
  # print("Overall Accuracy: %.4f"%(accuracy))
  return df

# Declaring polygons - Calculating Confusion Matrix

In [ ]:
px1_min, py1_min = (895-10,329-10)
px1_max, py1_max = (px1_min +20, py1_min +20)
p1 = [[px1_max,py1_max], [px1_min,py1_max], [px1_min,py1_min], [px1_max,py1_min],[px1_max,py1_max]]
px2_min, py2_min = (925-10,303-10)
px2_max, py2_max = (px2_min +20, py2_min +20)
p2 = [[px2_max,py2_max], [px2_min,py2_max], [px2_min,py2_min], [px2_max,py2_min],[px2_max,py2_max]]
px3_min, py3_min = (991-10,282-10)
px3_max, py3_max = (px3_min +20, py3_min +20)
p3 = [[px3_max,py3_max], [px3_min,py3_max], [px3_min,py3_min], [px3_max,py3_min],[px3_max,py3_max]]
px4_min, py4_min = (962-10,338-10)
px4_max, py4_max = (px4_min +20, py4_min +20)
p4 = [[px4_max,py4_max], [px4_min,py4_max], [px4_min,py4_min], [px4_max,py4_min],[px4_max,py4_max]]

px5_min, py5_min = (1243-10,195-10)
px5_max, py5_max = (px5_min +20, py5_min +20)
p5 = [[px5_max,py5_max], [px5_min,py5_max], [px5_min,py5_min], [px5_max,py5_min],[px5_max,py5_max]]
px6_min, py6_min = (1513-10,194-10)
px6_max, py6_max = (px6_min +20, py6_min +20)
p6 = [[px6_max,py6_max], [px6_min,py6_max], [px6_min,py6_min], [px6_max,py6_min],[px6_max,py6_max]]
px7_min, py7_min = (1247-10,313-10)
px7_max, py7_max = (px7_min +20, py7_min +20)
p7 = [[px7_max,py7_max], [px7_min,py7_max], [px7_min,py7_min], [px7_max,py7_min],[px7_max,py7_max]]
px8_min, py8_min = (1327-10,280-10)
px8_max, py8_max = (px8_min +20, py8_min +20)
p8 = [[px8_max,py8_max], [px8_min,py8_max], [px8_min,py8_min], [px8_max,py8_min],[px8_max,py8_max]]

px9_min, py9_min = (227-10,778-10)
px9_max, py9_max = (px9_min +20, py9_min +20)
p9 = [[px9_max,py9_max], [px9_min,py9_max], [px9_min,py9_min], [px9_max,py9_min],[px9_max,py9_max]]
px10_min, py10_min = (330-10,727-10)
px10_max, py10_max = (px10_min +20, py10_min +20)
p10 = [[px10_max,py10_max], [px10_min,py10_max], [px10_min,py10_min], [px10_max,py10_min],[px10_max,py10_max]]
px11_min, py11_min = (401-10,677-10)
px11_max, py11_max = (px11_min +20, py11_min +20)
p11 = [[px11_max,py11_max], [px11_min,py11_max], [px11_min,py11_min], [px11_max,py11_min],[px11_max,py11_max]]
px12_min, py12_min = (452-10,602-10)
px12_max, py12_max = (px12_min +20, py12_min +20)
p12 = [[px12_max,py12_max], [px12_min,py12_max], [px12_min,py12_min], [px12_max,py12_min],[px12_max,py12_max]]

px13_min, py13_min = (258-10,707-10)
px13_max, py13_max = (px13_min +20, py13_min +20)
p13 = [[px13_max,py13_max], [px13_min,py13_max], [px13_min,py13_min], [px13_max,py13_min],[px13_max,py13_max]]
px14_min, py14_min = (278-10,684-10)
px14_max, py14_max = (px14_min +20, py14_min +20)
p14 = [[px14_max,py14_max], [px14_min,py14_max], [px14_min,py14_min], [px14_max,py14_min],[px14_max,py14_max]]
px15_min, py15_min = (308-10,663-10)
px15_max, py15_max = (px15_min +20, py15_min +20)
p15 = [[px15_max,py15_max], [px15_min,py15_max], [px15_min,py15_min], [px15_max,py15_min],[px15_max,py15_max]]
px16_min, py16_min = (413-10,636-10)
px16_max, py16_max = (px16_min +20, py16_min +20)
p16 = [[px16_max,py16_max], [px16_min,py16_max], [px16_min,py16_min], [px16_max,py16_min],[px16_max,py16_max]]

px17_min, py17_min = (1016-10,354-10)
px17_max, py17_max = (px17_min +20, py17_min +20)
p17 = [[px17_max,py17_max], [px17_min,py17_max], [px17_min,py17_min], [px17_max,py17_min],[px17_max,py17_max]]
px18_min, py18_min = (976-10,409-10)
px18_max, py18_max = (px18_min +20, py18_min +20)
p18 = [[px18_max,py18_max], [px18_min,py18_max], [px18_min,py18_min], [px18_max,py18_min],[px18_max,py18_max]]
px19_min, py19_min = (1013-10,403-10)
px19_max, py19_max = (px19_min +20, py19_min +20)
p19 = [[px19_max,py19_max], [px19_min,py19_max], [px19_min,py19_min], [px19_max,py19_min],[px19_max,py19_max]]
px20_min, py20_min = (1050-10,409-10)
px20_max, py20_max = (px20_min +20, py20_min +20)
p20 = [[px20_max,py20_max], [px20_min,py20_max], [px20_min,py20_min], [px20_max,py20_min],[px20_max,py20_max]]

px21_min, py21_min = (652-10,509-10)
px21_max, py21_max = (px21_min +20, py21_min +20)
p21 = [[px21_max,py21_max], [px21_min,py21_max], [px21_min,py21_min], [px21_max,py21_min],[px21_max,py21_max]]
px22_min, py22_min = (673-10,530-10)
px22_max, py22_max = (px22_min +20, py22_min +20)
p22 = [[px22_max,py22_max], [px22_min,py22_max], [px22_min,py22_min], [px22_max,py22_min],[px22_max,py22_max]]
px23_min, py23_min = (580-10,581-10)
px23_max, py23_max = (px23_min +20, py23_min +20)
p23 = [[px23_max,py23_max], [px23_min,py23_max], [px23_min,py23_min], [px23_max,py23_min],[px23_max,py23_max]]
px24_min, py24_min = (654-10,564-10)
px24_max, py24_max = (px24_min +20, py24_min +20)
p24 = [[px24_max,py24_max], [px24_min,py24_max], [px24_min,py24_min], [px24_max,py24_min],[px24_max,py24_max]]

px25_min, py25_min = (890-10,389-10)
px25_max, py25_max = (px25_min +20, py25_min +20)
p25 = [[px25_max,py25_max], [px25_min,py25_max], [px25_min,py25_min], [px25_max,py25_min],[px25_max,py25_max]]
px26_min, py26_min = (995-10,264-10)
px26_max, py26_max = (px26_min +20, py26_min +20)
p26 = [[px26_max,py26_max], [px26_min,py26_max], [px26_min,py26_min], [px26_max,py26_min],[px26_max,py26_max]]
px27_min, py27_min = (1131-10,312-10)
px27_max, py27_max = (px27_min +20, py27_min +20)
p27 = [[px27_max,py27_max], [px27_min,py27_max], [px27_min,py27_min], [px27_max,py27_min],[px27_max,py27_max]]
px28_min, py28_min = (1075-10,390-10)
px28_max, py28_max = (px28_min +20, py28_min +20)
p28 = [[px28_max,py28_max], [px28_min,py28_max], [px28_min,py28_min], [px28_max,py28_min],[px28_max,py28_max]]

px29_min, py29_min = (876-10,333-10)
px29_max, py29_max = (px29_min +20, py29_min +20)
p29 = [[px29_max,py29_max], [px29_min,py29_max], [px29_min,py29_min], [px29_max,py29_min],[px29_max,py29_max]]
px30_min, py30_min = (760-10,411-10)
px30_max, py30_max = (px30_min +20, py30_min +20)
p30 = [[px30_max,py30_max], [px30_min,py30_max], [px30_min,py30_min], [px30_max,py30_min],[px30_max,py30_max]]
px31_min, py31_min = (991-10,369-10)
px31_max, py31_max = (px31_min +20, py31_min +20)
p31 = [[px31_max,py31_max], [px31_min,py31_max], [px31_min,py31_min], [px31_max,py31_min],[px31_max,py31_max]]
px32_min, py32_min = (1034-10,386-10)
px32_max, py32_max = (px32_min +20, py32_min +20)
p32 = [[px32_max,py32_max], [px32_min,py32_max], [px32_min,py32_min], [px32_max,py32_min],[px32_max,py32_max]]

px33_min, py33_min = (304-10,717-10)
px33_max, py33_max = (px33_min +20, py33_min +20)
p33 = [[px33_max,py33_max], [px33_min,py33_max], [px33_min,py33_min], [px33_max,py33_min],[px33_max,py33_max]]
px34_min, py34_min = (354-10,669-10)
px34_max, py34_max = (px34_min +20, py34_min +20)
p34 = [[px34_max,py34_max], [px34_min,py34_max], [px34_min,py34_min], [px34_max,py34_min],[px34_max,py34_max]]
px35_min, py35_min = (786-10,471-10)
px35_max, py35_max = (px35_min +20, py35_min +20)
p35 = [[px35_max,py35_max], [px35_min,py35_max], [px35_min,py35_min], [px35_max,py35_min],[px35_max,py35_max]]
px36_min, py36_min = (1349-10,108-10)
px36_max, py36_max = (px36_min +20, py36_min +20)
p36 = [[px36_max,py36_max], [px36_min,py36_max], [px36_min,py36_min], [px36_max,py36_min],[px36_max,py36_max]]

px37_min, py37_min = (308-10,650-10)
px37_max, py37_max = (px37_min +20, py37_min +20)
p37 = [[px37_max,py37_max], [px37_min,py37_max], [px37_min,py37_min], [px37_max,py37_min],[px37_max,py37_max]]
px38_min, py38_min = (440,505)
px38_max, py38_max = (px38_min +20, py38_min +20)
p38 = [[px38_max,py38_max], [px38_min,py38_max], [px38_min,py38_min], [px38_max,py38_min],[px38_max,py38_max]]
px39_min, py39_min = (1314-10,92-10)
px39_max, py39_max = (px39_min +20, py39_min +20)
p39 = [[px39_max,py39_max], [px39_min,py39_max], [px39_min,py39_min], [px39_max,py39_min],[px39_max,py39_max]]
px40_min, py40_min = (1115-10,194-10)
px40_max, py40_max = (px40_min +20, py40_min +20)
p40 = [[px40_max,py40_max], [px40_min,py40_max], [px40_min,py40_min], [px40_max,py40_min],[px40_max,py40_max]]

px41_min, py41_min = (242-10,723-10)
px41_max, py41_max = (px41_min +20, py41_min +20)
p41 = [[px41_max,py41_max], [px41_min,py41_max], [px41_min,py41_min], [px41_max,py41_min],[px41_max,py41_max]]
# px42_min, py42_min = (797-10,415-10)
# px42_max, py42_max = (px42_min +20, py42_min +20)
# p42 = [[px42_max,py42_max], [px42_min,py42_max], [px42_min,py42_min], [px42_max,py42_min],[px42_max,py42_max]]
px43_min, py43_min = (1137-10,264-10)
px43_max, py43_max = (px43_min +20, py43_min +20)
p43 = [[px43_max,py43_max], [px43_min,py43_max], [px43_min,py43_min], [px43_max,py43_min],[px43_max,py43_max]]
# px44_min, py44_min = (1648-10,107-10)
# px44_max, py44_max = (px44_min +20, py44_min +20)
# p44 = [[px44_max,py44_max], [px44_min,py44_max], [px44_min,py44_min], [px44_max,py44_min],[px44_max,py44_max]]

px45_min, py45_min = (644-10,355-10)
px45_max, py45_max = (px45_min +20, py45_min +20)
p45 = [[px45_max,py45_max], [px45_min,py45_max], [px45_min,py45_min], [px45_max,py45_min],[px45_max,py45_max]]
# px46_min, py46_min = (722-10,610-10)
# px46_max, py46_max = (px46_min +20, py46_min +20)
# p46 = [[px46_max,py46_max], [px46_min,py46_max], [px46_min,py46_min], [px46_max,py46_min],[px46_max,py46_max]]
# px47_min, py47_min = (1216-10,389-10)
# px47_max, py47_max = (px47_min +20, py47_min +20)
# p47 = [[px47_max,py47_max], [px47_min,py47_max], [px47_min,py47_min], [px47_max,py47_min],[px47_max,py47_max]]
# px48_min, py48_min = (1079-10,173-10)
# px48_max, py48_max = (px48_min +20, py48_min +20)
# p48 = [[px48_max,py48_max], [px48_min,py48_max], [px48_min,py48_min], [px48_max,py48_min],[px48_max,py48_max]]



df1	  =	calc_conf_mat(	px1_min		,	px1_max		,	py1_min		,	py1_max		)
df2	  =	calc_conf_mat(	px2_min		,	px2_max		,	py2_min		,	py2_max		)
df3	  =	calc_conf_mat(	px3_min		,	px3_max		,	py3_min		,	py3_max		)
df4	  =	calc_conf_mat(	px4_min		,	px4_max		,	py4_min		,	py4_max		)
df5	  =	calc_conf_mat(	px5_min		,	px5_max		,	py5_min		,	py5_max		)
df6	  =	calc_conf_mat(	px6_min		,	px6_max		,	py6_min		,	py6_max		)
df7	  =	calc_conf_mat(	px7_min		,	px7_max		,	py7_min		,	py7_max		)
df8	  =	calc_conf_mat(	px8_min		,	px8_max		,	py8_min		,	py8_max		)
df9	  =	calc_conf_mat(	px9_min		,	px9_max		,	py9_min		,	py9_max		)
df10	=	calc_conf_mat(	px10_min	,	px10_max	,	py10_min	,	py10_max	)
df11	=	calc_conf_mat(	px11_min	,	px11_max	,	py11_min	,	py11_max	)
df12	=	calc_conf_mat(	px12_min	,	px12_max	,	py12_min	,	py12_max	)
df13	=	calc_conf_mat(	px13_min	,	px13_max	,	py13_min	,	py13_max	)
df14	=	calc_conf_mat(	px14_min	,	px14_max	,	py14_min	,	py14_max	)
df15	=	calc_conf_mat(	px15_min	,	px15_max	,	py15_min	,	py15_max	)
df16	=	calc_conf_mat(	px16_min	,	px16_max	,	py16_min	,	py16_max	)
df17	=	calc_conf_mat(	px17_min	,	px17_max	,	py17_min	,	py17_max	)
df18	=	calc_conf_mat(	px18_min	,	px18_max	,	py18_min	,	py18_max	)
df19	=	calc_conf_mat(	px19_min	,	px19_max	,	py19_min	,	py19_max	)
df20	=	calc_conf_mat(	px20_min	,	px20_max	,	py20_min	,	py20_max	)
df21	=	calc_conf_mat(	px21_min	,	px21_max	,	py21_min	,	py21_max	)
df22	=	calc_conf_mat(	px22_min	,	px22_max	,	py22_min	,	py22_max	)
df23	=	calc_conf_mat(	px23_min	,	px23_max	,	py23_min	,	py23_max	)
df24	=	calc_conf_mat(	px24_min	,	px24_max	,	py24_min	,	py24_max	)
df25	=	calc_conf_mat(	px25_min	,	px25_max	,	py25_min	,	py25_max	)
df26	=	calc_conf_mat(	px26_min	,	px26_max	,	py26_min	,	py26_max	)
df27	=	calc_conf_mat(	px27_min	,	px27_max	,	py27_min	,	py27_max	)
df28	=	calc_conf_mat(	px28_min	,	px28_max	,	py28_min	,	py28_max	)
df29	=	calc_conf_mat(	px29_min	,	px29_max	,	py29_min	,	py29_max	)
df30	=	calc_conf_mat(	px30_min	,	px30_max	,	py30_min	,	py30_max	)
df31	=	calc_conf_mat(	px31_min	,	px31_max	,	py31_min	,	py31_max	)
df32	=	calc_conf_mat(	px32_min	,	px32_max	,	py32_min	,	py32_max	)
df33	=	calc_conf_mat(	px33_min	,	px33_max	,	py33_min	,	py33_max	)
df34	=	calc_conf_mat(	px34_min	,	px34_max	,	py34_min	,	py34_max	)
df35	=	calc_conf_mat(	px35_min	,	px35_max	,	py35_min	,	py35_max	)
df36	=	calc_conf_mat(	px36_min	,	px36_max	,	py36_min	,	py36_max	)
df37	=	calc_conf_mat(	px37_min	,	px37_max	,	py37_min	,	py37_max	)
df38	=	calc_conf_mat(	px38_min	,	px38_max	,	py38_min	,	py38_max	)
df39	=	calc_conf_mat(	px39_min	,	px39_max	,	py39_min	,	py39_max	)
df40	=	calc_conf_mat(	px40_min	,	px40_max	,	py40_min	,	py40_max	)
df41	=	calc_conf_mat(	px41_min	,	px41_max	,	py41_min	,	py41_max	)
# df42	=	calc_conf_mat(	px42_min	,	px42_max	,	py42_min	,	py42_max	)
df43	=	calc_conf_mat(	px43_min	,	px43_max	,	py43_min	,	py43_max	)
# df44	=	calc_conf_mat(	px44_min	,	px44_max	,	py44_min	,	py44_max	)
df45	=	calc_conf_mat(	px45_min	,	px45_max	,	py45_min	,	py45_max	)
# df46	=	calc_conf_mat(	px46_min	,	px46_max	,	py46_min	,	py46_max	)
# df47	=	calc_conf_mat(	px47_min	,	px47_max	,	py47_min	,	py47_max	)
# df48	=	calc_conf_mat(	px48_min	,	px48_max	,	py48_min	,	py48_max	)
# έχω βγάλει από την πρόσθεση τα παραπάνω
df_final = df1 + 	df2 + 	df3 + 	df4 + 	df5 + 	df6 + 	df7 + 	df8 + 	df9 + 	df10 + 	df11 + 	df12 + 	df13 + 	df14 + 	df15 + 	df16 + 	df17 + 	df18 + 	df19 + 	df20 + 	df21 + 	df22 + 	df23 + 	df24 + 	df25 + 	df26 + 	df27 + 	df28 + 	df29 + 	df30 + 	df31 + 	df32 + 	df33 + 	df34 + 	df35 + 	df36 + 	df37 + 	df38 + 	df39 + 	df40 + 	df41 + 	 	df43 + 	 	df45  
df_final


# Omission/Commission Errors - Total Accuracy

In [ ]:
counter = 0
diagonal = 0
for i in df_final:
  if i == 'Total':
    break
  else:
    diagonal = diagonal + df_final[i][i]

  if df_final[i][-1] == 0:
    commission_error = 0
  else:
    commission_error = (df_final[i][-1] - df_final[i][counter]) / df_final[i][-1]

  
  if df_final.loc[i][-1] == 0:
    omission_error = 0
  else:
    omission_error = (df_final.loc[i][-1] - df_final.loc[i][counter]) / df_final.loc[i][-1]

  
  counter += 1
  
  print('%s\nOmission Error: %.1f %% , Commission Error: %.1f %% \n'%(i, omission_error*100,commission_error*100))
accuracy = diagonal/df_final['Total']['Total']
print("Overall Accuracy: %.1f %%"%(accuracy*100))
# http://gis.humboldt.edu/OLM/Courses/GSP_216_Online/lesson6-2/metrics.html

# Structural Similarity Index

In [ ]:
temp = y_valid_final[0][:,:,-1]
indices = temp<1
for i in y_valid_final:
  tim1 = y_valid_final[i].astype(float)
  tim2 = result_final[i].astype(float)
  total = skimage.metrics.structural_similarity(tim1, tim2)
  tim3 = y_valid_final[i][indices,:].astype(float)
  tim4 = result_final[i][indices,:].astype(float)
  inner = skimage.metrics.structural_similarity(tim3, tim4)
  print("Image",i+1)
  print("Total similarity: %.4f\nInner similarity: %.4f\n"%(total,inner))

# Plot image with Conf. Matrix polygons

In [ ]:
im_result_colors = np.zeros((im_result.shape[0],im_result.shape[1]))
im_y_valid_colors = np.zeros((im_y_valid.shape[0],im_y_valid.shape[1]))
for k in range(im_result.shape[2]):
  im_result_colors = im_result_colors + im_result[:,:,k]*k
  im_y_valid_colors = im_y_valid_colors + im_y_valid[:,:,k]*k

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("",
[
matplotlib.colors.to_rgb([1,0,0]), # 'Disc. Urban Fabric'
matplotlib.colors.to_rgb([0.8,0.8,0.8]), # 'Bare Rocks'
matplotlib.colors.to_rgb([0,0.650980392,0]), # 'Coniferous Forest'
matplotlib.colors.to_rgb([0.301960784,1,0]), # 'Mixed Forest'
matplotlib.colors.to_rgb([0.8,0.949019608,0.301960784]), # 'Natural Grasslands'
matplotlib.colors.to_rgb([0.901960784,0.650980392,0]), # 'Olive Groves'
matplotlib.colors.to_rgb([0.901960784,0.901960784,0.301960784]), # 'Pastures'
matplotlib.colors.to_rgb([0.901960784,0.8,0.301960784]), # 'Principally Agriculture'
matplotlib.colors.to_rgb([0.650980392,0.901960784,0.301960784]), # 'Schlerophyllus Vegetation'
# matplotlib.colors.to_rgb([0.901960784,0.949019608,1]), # 'Sea'
"lightblue", # 'Sea'
matplotlib.colors.to_rgb([0.8,1,0.8]), # 'Sparsely Veg. Areas'
matplotlib.colors.to_rgb([0,0,0]) # 'No Data'
])
fig = plt.figure(1, figsize=(20, 20))
ax = plt.gca()
im = ax.imshow(im_result_colors, cmap=cmap)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im, cax=cax)

# px1_min,px1_max,py1_min,py1_max = (900,1000,300,430)
# px2_min,px2_max,py2_min,py2_max = (550,700,550,600)
# px3_min,px3_max,py3_min,py3_max = (1350,1450,150,250)
# p1 = [[px1_max,py1_max], [px1_min,py1_max], [px1_min,py1_min], [px1_max,py1_min],[px1_max,py1_max]] # 900< y < 1000     300 < x < 430
# p2 = [[px2_max,py2_max], [px2_min,py2_max], [px2_min,py2_min], [px2_max,py2_min],[px2_max,py2_max]] # 550< y < 700     550 < x < 600
# p3 = [[px3_max,py3_max], [px3_min,py3_max], [px3_min,py3_min], [px3_max,py3_min],[px3_max,py3_max]] # 1350 < y < 1450     150 < x < 250

xp1	  ,	yp1	  =	zip(	*p1	  )
xp2	  ,	yp2	  =	zip(	*p2	  )
xp3	  ,	yp3	  =	zip(	*p3	  )
xp4	  ,	yp4	  =	zip(	*p4	  )
xp5	  ,	yp5	  =	zip(	*p5	  )
xp6	  ,	yp6	  =	zip(	*p6	  )
xp7	  ,	yp7	  =	zip(	*p7	  )
xp8	  ,	yp8	  =	zip(	*p8	  )
xp9	  ,	yp9	  =	zip(	*p9	  )
xp10	,	yp10	=	zip(	*p10	)
xp11	,	yp11	=	zip(	*p11	)
xp12	,	yp12	=	zip(	*p12	)
xp13	,	yp13	=	zip(	*p13	)
xp14	,	yp14	=	zip(	*p14	)
xp15	,	yp15	=	zip(	*p15	)
xp16	,	yp16	=	zip(	*p16	)
xp17	,	yp17	=	zip(	*p17	)
xp18	,	yp18	=	zip(	*p18	)
xp19	,	yp19	=	zip(	*p19	)
xp20	,	yp20	=	zip(	*p20	)
xp21	,	yp21	=	zip(	*p21	)
xp22	,	yp22	=	zip(	*p22	)
xp23	,	yp23	=	zip(	*p23	)
xp24	,	yp24	=	zip(	*p24	)
xp25	,	yp25	=	zip(	*p25	)
xp26	,	yp26	=	zip(	*p26	)
xp27	,	yp27	=	zip(	*p27	)
xp28	,	yp28	=	zip(	*p28	)
xp29	,	yp29	=	zip(	*p29	)
xp30	,	yp30	=	zip(	*p30	)
xp31	,	yp31	=	zip(	*p31	)
xp32	,	yp32	=	zip(	*p32	)
xp33	,	yp33	=	zip(	*p33	)
xp34	,	yp34	=	zip(	*p34	)
xp35	,	yp35	=	zip(	*p35	)
xp36	,	yp36	=	zip(	*p36	)
xp37	,	yp37	=	zip(	*p37	)
xp38	,	yp38	=	zip(	*p38	)
xp39	,	yp39	=	zip(	*p39	)
xp40	,	yp40	=	zip(	*p40	)
xp41	,	yp41	=	zip(	*p41	)
# xp42	,	yp42	=	zip(	*p42	)
xp43	,	yp43	=	zip(	*p43	)
# xp44	,	yp44	=	zip(	*p44	)
xp45	,	yp45	=	zip(	*p45	)
# xp46	,	yp46	=	zip(	*p46	)
# xp47	,	yp47	=	zip(	*p47	)
# xp48	,	yp48	=	zip(	*p48	)

ax.plot(	xp1	,	yp1	,	'w-'	)
ax.plot(	xp2	,	yp2	,	'w-'	)
ax.plot(	xp3	,	yp3	,	'w-'	)
ax.plot(	xp4	,	yp4	,	'w-'	)
ax.plot(	xp5	,	yp5	,	'w-'	)
ax.plot(	xp6	,	yp6	,	'w-'	)
ax.plot(	xp7	,	yp7	,	'w-'	)
ax.plot(	xp8	,	yp8	,	'w-'	)
ax.plot(	xp9	,	yp9	,	'w-'	)
ax.plot(	xp10	,	yp10	,	'w-'	)
ax.plot(	xp11	,	yp11	,	'w-'	)
ax.plot(	xp12	,	yp12	,	'w-'	)
ax.plot(	xp13	,	yp13	,	'w-'	)
ax.plot(	xp14	,	yp14	,	'w-'	)
ax.plot(	xp15	,	yp15	,	'w-'	)
ax.plot(	xp16	,	yp16	,	'w-'	)
ax.plot(	xp17	,	yp17	,	'w-'	)
ax.plot(	xp18	,	yp18	,	'w-'	)
ax.plot(	xp19	,	yp19	,	'w-'	)
ax.plot(	xp20	,	yp20	,	'w-'	)
ax.plot(	xp21	,	yp21	,	'w-'	)
ax.plot(	xp22	,	yp22	,	'w-'	)
ax.plot(	xp23	,	yp23	,	'w-'	)
ax.plot(	xp24	,	yp24	,	'w-'	)
ax.plot(	xp25	,	yp25	,	'w-'	)
ax.plot(	xp26	,	yp26	,	'w-'	)
ax.plot(	xp27	,	yp27	,	'w-'	)
ax.plot(	xp28	,	yp28	,	'w-'	)
ax.plot(	xp29	,	yp29	,	'w-'	)
ax.plot(	xp30	,	yp30	,	'w-'	)
ax.plot(	xp31	,	yp31	,	'w-'	)
ax.plot(	xp32	,	yp32	,	'w-'	)
ax.plot(	xp33	,	yp33	,	'w-'	)
ax.plot(	xp34	,	yp34	,	'w-'	)
ax.plot(	xp35	,	yp35	,	'w-'	)
ax.plot(	xp36	,	yp36	,	'w-'	)
ax.plot(	xp37	,	yp37	,	'w-'	)
ax.plot(	xp38	,	yp38	,	'w-'	)
ax.plot(	xp39	,	yp39	,	'w-'	)
ax.plot(	xp40	,	yp40	,	'w-'	)
ax.plot(	xp41	,	yp41	,	'w-'	)
# ax.plot(	xp42	,	yp42	,	'w-'	)
ax.plot(	xp43	,	yp43	,	'w-'	)
# ax.plot(	xp44	,	yp44	,	'w-'	)
ax.plot(	xp45	,	yp45	,	'w-'	)
# ax.plot(	xp46	,	yp46	,	'w-'	)
# ax.plot(	xp47	,	yp47	,	'w-'	)
# ax.plot(	xp48	,	yp48	,	'w-'	)



plot_name = data_dir[42:-1]+'_plot_4.png'
plt.savefig(plot_name, bbox_inches='tight',dpi=600)
plt.show()


# plt.figure(figsize=(15,15))
# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","lightgray","darkgreen","lime","greenyellow","darkgoldenrod","brown","darkorange","yellowgreen","lightblue", "palegreen"])
# plt.imshow(im_result_colors, cmap=cmap)
# plt.colorbar()

#  Plotting validation data predictions

In [ ]:
for i in y_valid_final:
  im_result = result_final[i]
  im_y_valid = y_valid_final[i]

  im_result_colors = np.zeros((im_result.shape[0],im_result.shape[1]))
  im_y_valid_colors = np.zeros((im_y_valid.shape[0],im_y_valid.shape[1]))
  for k in range(im_result.shape[2]):
    im_result_colors = im_result_colors + im_result[:,:,k]*k
    im_y_valid_colors = im_y_valid_colors + im_y_valid[:,:,k]*k

  fig = plt.figure(1, figsize=(15, 15))
  ax = plt.gca()
  im = ax.imshow(im_result_colors, cmap=cmap)
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.1)
  plt.colorbar(im, cax=cax)
  plt.grid(b=None)
  plot_name = data_dir[42:-1]+'_validation_image'+str(i+1)+'_plot_4.png'
  plt.savefig(plot_name, bbox_inches='tight',dpi=600)
  plt.close(fig)


# Calculating weight significance

In [ ]:
filters, biases = model.layers[1].get_weights()
print(filters.shape)
for i in range(filters.shape[2]):
  print(np.sum(abs(filters[:,:,i,:])))